In [1]:
import pandas as pd
import requests

In [6]:
def download_carga_horaria(ano_inicio: int, ano_fim: int, printer: bool=False) -> pd.DataFrame:
    """Função para fazer download dos dados de carga elétrica por subsistema no período de referência em base horária.
       Página: https://dados.ons.org.br/dataset/curva-carga

    Args:
        ano_inicio (int): ano inicial de extração.
        ano_fim (int): ano final de extração.
        printer(bool): rastreia ano que está sendo extraído com print em tela.

    Returns:
        pd.DataFrame: Pandas DataFrame com dados de carga horária entre ano_inicio e ano_fim, tendo como índice a data-hora.
    """

    url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{}.csv"
    # verificar se anos inicial e final estão disponíveis
    get0 = requests.get(url.format(ano_inicio)).status_code # verify = False (autenticação)
    getn = requests.get(url.format(ano_fim)).status_code 
    if (get0 == 200) and (getn == 200): # 200: página (ano) disponível
        # concatenar arquivos de cada ano em um único dataframe
        df = pd.DataFrame()
        for ano in range(ano_inicio, ano_fim + 1):
            if printer:
                print(f"Lendo ano {ano}...")
            df2 = pd.read_csv(url.format(ano), sep = ";")
            df = pd.concat([df, df2])
        df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
        df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = '%Y-%m-%d %H:%M:%S')
        df.sort_values(by = "date", inplace = True)
        df.set_index("date", inplace=True)
        return df
    else:
       print("Ano não disponível.")

In [7]:
df = download_carga_horaria(2000, 2023, printer=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


In [8]:
df

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01 00:00:00,N,NORTE,2373.7
2000-01-01 00:00:00,NE,NORDESTE,5340.2
2000-01-01 00:00:00,S,SUL,5777.0
2000-01-01 00:00:00,SE,SUDESTE,21183.0
2000-01-01 01:00:00,N,NORTE,2331.6
...,...,...,...
2023-02-26 19:00:00,S,SUL,0.0
2023-02-26 20:00:00,S,SUL,0.0
2023-02-26 21:00:00,S,SUL,0.0


In [13]:
def download_carga_diaria(ano_inicio: int, ano_fim: int, printer: bool=False):
    """ Função para fazer download dos dados de carga elétrica por subsistema no período de referência em base diária."""

    url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{}.csv"

    # verificar se anos inicial e final estão disponíveis
    get0 = requests.get(url.format(ano_inicio)).status_code # verify = False (autenticação)
    getn = requests.get(url.format(ano_fim)).status_code 
    if (get0 == 200) and (getn == 200): # 200: página (ano) disponível

        # concatenar arquivos de cada ano em um único dataframe
        df = pd.DataFrame()
        for ano in range(ano_inicio, ano_fim + 1):
            if printer:
                print(f"Lendo ano {ano}...")
            df2 = pd.read_csv(url.format(ano), sep = ";")
            df = pd.concat([df, df2])
        df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
        df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = '%Y-%m-%d')
        df.sort_values(by = "date", inplace = True)
        df.set_index("date", inplace=True)
        return df
    
    else:
       print("Ano não disponível.")

In [14]:
df = download_carga_diaria(2000, 2023, printer=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


In [15]:
df

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01,N,NORTE,2243.512500
2000-01-01,NE,NORDESTE,4646.370833
2000-01-01,S,SUL,4800.650000
2000-01-01,SE,SUDESTE,19045.995833
2000-01-02,N,NORTE,2259.808333
...,...,...,...
2023-02-25,NE,NORDESTE,NaN
2023-02-26,NE,NORDESTE,NaN
2023-02-26,S,SUL,NaN


In [3]:
class ons_data:
    def __init__(self, freq, ano_inicio, ano_fim):
        self.freq = freq
        self.ano_inicio = ano_inicio
        self.ano_fim = ano_fim
        self.data_dir = "../../../data/"

    def read(self):
        if self.freq == "hourly":
            path = "".join([self.data_dir,"hourly_load.csv"])
        elif self.freq == "daily":
            path = "".join([self.data_dir,"daily_load.csv"])
        df = pd.read_csv(path)
        return df

    def update(self, printer=False, write=False):
        if self.freq == "hourly":
            url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{}.csv"
            date_format = "%Y-%m-%d %H:%M:%S"
        elif self.freq == "daily":
            url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{}.csv"
            date_format = "%Y-%m-%d"
        else:
            raise Exception("Frequência não reconhecida. Utilize 'hourly' ou 'daily'.")
        get0 = requests.get(url.format(self.ano_inicio)).status_code # verify = False (autenticação)
        getn = requests.get(url.format(self.ano_fim)).status_code 
        if (get0 == 200) and (getn == 200): # 200: página (ano) disponível
            # concatenar arquivos de cada ano em um único dataframe
            df = pd.DataFrame()
            for ano in range(self.ano_inicio, self.ano_fim + 1):
                if printer:
                    print(f"Lendo ano {ano}...")
                df2 = pd.read_csv(url.format(ano), sep = ";")
                df = pd.concat([df, df2])
            df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
            df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = date_format)
            df.sort_values(by = "date", inplace = True)
            df.set_index("date", inplace=True)
            if write:
                full_path = "".join([self.data_dir,f"{self.freq}_load.csv"])
                df.to_csv(full_path)
            return df
        else:
            print("Ano não disponível.")

In [8]:
data = ons_data('hourly', 2000, 2023)

In [9]:
df = data.update(printer=True, write=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


C:\Users\user\AppData\Local\Temp\ipykernel_2940\2898127356.py:36: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = date_format)


In [6]:
df

,date,id_reg,desc_reg,load_mwmed
0,2000-01-01 00:00:00,N,NORTE,2243.512500
1,2000-01-01 00:00:00,NE,NORDESTE,4646.370833
2,2000-01-01 00:00:00,S,SUL,4800.650000
3,2000-01-01 00:00:00,SE,SUDESTE,19045.995833
4,2000-01-02 00:00:00,N,NORTE,2259.808333
...,...,...,...,...
33828,2023-02-25,NE,NORDESTE,NaN
33829,2023-02-26,NE,NORDESTE,NaN
33830,2023-02-26,S,SUL,NaN
33831,2023-02-26,N,NORTE,NaN
